## Spectral Results Inspection

This notebook plots and compares feature boundaries chosen by different team members.

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import yaml
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot
from bokeh.models import Span, Range1d
from bokeh.plotting import figure
from matplotlib import pyplot as plt
from scipy.ndimage.filters import median_filter
from sndata.csp import DR1, DR3

base_dir = Path('.').resolve().parent
sys.path.insert(0, str(base_dir))

from scripts.run_csp import pre_process, get_csp_t0
from spec_analysis.spectra import Spectrum


In [ ]:
output_notebook()


In [ ]:
dr1 = DR1()
dr1.download_module_data()

dr3 = DR3()
dr3.download_module_data()

# Output directory for figures
fig_dir = Path('./figs/classification')
fig_dir.mkdir(exist_ok=True, parents=True)

# Project data
results_dir = Path('.').resolve().parent / 'results'
config_path = Path('../app_config.yml')
with config_path.open() as config_file:
    config_data = yaml.load(config_file, Loader=yaml.SafeLoader)


We create a combined dataframe of the available spectroscopic measurements. Columns are suffixed with the inspector's name.

In [ ]:
def read_in_pipeline_result(path):
    """Read pEW values from analysis pipline file
    
    Adds columns for Branch classifications determined by the
    measured pEW values and spectral subtypes determined from 
    CSP DR1.
    
    Args:
        path (str): Path of the file to read
        
    Returns:
        A pandas Dataframe indexed by feat_name and obj_id
    """
    
    df = pd.read_csv(path, index_col=['obj_id', 'feat_name', 'time'])

    # Add phases using CSP DR3 t0 values
    obj_id = df.index.get_level_values(0)
    time = df.index.get_level_values(2)
    
    phase = np.array([get_csp_t0(oid) for oid in obj_id])
    df['phase'] = phase - time

    # Add Branch style classifications
    pw = pd.DataFrame({
        'pW6': df.xs('pW6', level=1).pew, 
        'pW7': df.xs('pW7', level=1).pew}
    ).dropna()
    
    # Add spectral subtypes
    csp_table_2 = dr3.load_table(2)
    subtypes = pd.DataFrame({'spec_type': csp_table_2['Subtype1']}, index=csp_table_2['SN'])
    df = df.join(subtypes, on='obj_id')
    
    return df


In [ ]:
ella = read_in_pipeline_result(results_dir / 'ella_csp.csv')
emily = read_in_pipeline_result(results_dir / 'emily_csp.csv')
anish = read_in_pipeline_result(results_dir / 'anish_csp.csv')

combined = ella.join(emily, lsuffix='_ella', rsuffix='_emily')
combined = combined.join(anish.add_suffix('_anish'))

combined.head()


In [ ]:
def plot_spectral_measurements(obj_id, time, feature, results=combined):
    """Plot an interactive comparison of estimated feature boundaries
    
    Args:
        obj_id        (str): The object to plot
        time        (float): Observation time of the desired object's spectrum
        feature       (str): Name of the feature to inspect (e.g., 'pW1')
        results (DataFrame): Dataframe of inspection results
    """
    
    # Load object data
    data = dr1.get_data_for_id(obj_id)
    processed_data = pre_process(data)
    processed_data = processed_data[processed_data['time'] == time]
    spectrum = Spectrum(
        processed_data['wavelength'], 
        processed_data['flux'],
        processed_data.meta['ra'],
        processed_data.meta['dec'],
        processed_data.meta['z'],
    )
    spectrum.prepare_spectrum(bin_size=10)

    # Create a seperate subplot for each inspector
    sub_size = (800, 400)
    title = f'{obj_id} - {time} - {feature}'
    s1 = figure(plot_width=sub_size[0], plot_height=sub_size[1], title=title)
    s2 = figure(plot_width=sub_size[0], plot_height=sub_size[1], x_range=s1.x_range, y_range=s1.y_range)
    s3 = figure(plot_width=sub_size[0], plot_height=sub_size[1], x_range=s1.x_range,  y_range=s1.y_range)

    # Instantiate plotting args
    inspectors = ('ella', 'emily', 'anish')
    obj_results = results.loc[obj_id, feature, time]
    lower_blue = config_data['features'][feature]['lower_blue']
    upper_blue = config_data['features'][feature]['upper_blue']
    lower_red = config_data['features'][feature]['lower_red']
    upper_red = config_data['features'][feature]['upper_red']

    for fig, insp in zip((s1, s2, s3), inspectors):
        # Format figure
        fig.yaxis.axis_label = insp
        fig.y_range = Range1d(0, 1.1 * max(spectrum.rest_flux))

        # Plot object data and auto boundaries
        fig.line(spectrum.rest_wave, spectrum.rest_flux, alpha=0.3, color='black')
        fig.line(spectrum.rest_wave, spectrum.bin_flux, alpha=0.8, color='black')
        fig.harea([lower_blue, lower_blue], [upper_blue, upper_blue], [-1, 2], alpha=.1)
        fig.harea([lower_red, lower_red], [upper_red, upper_red], [-1, 2], alpha=.1, color='red')

        measured_start = obj_results[f'feat_start_{insp}']
        measured_end = obj_results[f'feat_end_{insp}']
        if np.isnan(measured_start):
            continue

        vline1 = Span(location=measured_start, dimension='height', line_color='red', line_width=1)
        vline2 = Span(location=measured_end, dimension='height', line_color='red', line_width=1)
        fig.renderers.extend([vline1, vline2])

    p = gridplot([[s1], [s2], [s3]])
    show(p)


In [ ]:
plot_spectral_measurements('2004ef', 2453255.66, 'pW2')
